In [1]:
!pip install sentence_transformers
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.2 MB/s eta 0:00:00


In [2]:
import torch
from sentence_transformers import SentenceTransformer, CrossEncoder
import faiss
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
sbert_model = SentenceTransformer('paraphrase-MiniLM-L6-v2', device=device)
cross_encoder = CrossEncoder('cross-encoder/stsb-roberta-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [4]:
dog_data = pd.read_csv('cleaned_dataset.csv')

In [5]:
numeric_features = ['Average Cost', 'Average Height', 'Average Weight']
scaler = StandardScaler()
dog_data[numeric_features] = scaler.fit_transform(dog_data[numeric_features])

In [6]:
# Обработка shedding данных в зависимости от пользовательского ввода
def process_shedding(dog_data, shedding_input):
    dog_data['Sheds Little'] = 0
    dog_data['Sheds'] = 0
    dog_data['Sheds A Lot'] = 0

    if shedding_input == 'Sheds Little':
        dog_data['Sheds Little'] = 1
    elif shedding_input == 'Sheds':
        dog_data['Sheds'] = 1
    elif shedding_input == 'Sheds A Lot':
        dog_data['Sheds A Lot'] = 1
    elif shedding_input == 'No Sheds':
        dog_data['Sheds Little'] = 0
        dog_data['Sheds'] = 0
        dog_data['Sheds A Lot'] = 0

    return dog_data

In [7]:
def denormalize(value, column_name):
    mean = scaler.mean_[numeric_features.index(column_name)]
    std = scaler.scale_[numeric_features.index(column_name)]
    return value * std + mean

In [8]:
def denormalize_features(dog):
    original_cost = denormalize(dog['Average Cost'], 'Average Cost')
    original_height = denormalize(dog['Average Height'], 'Average Height')
    original_weight = denormalize(dog['Average Weight'], 'Average Weight')
    return original_cost, original_height, original_weight

In [9]:
def relevance_to_percentage(distances):
    relevance_scores = 1 / (distances + 1e-5)
    relevance_normalized = relevance_scores / np.max(relevance_scores)
    return relevance_normalized

In [10]:
def encode_text(text, model):
    if not isinstance(text, str):
        text = str(text)
    embeddings = model.encode(text, convert_to_tensor=True).cpu().numpy()
    return embeddings

In [11]:
dog_data['embedding'] = dog_data['Description'].apply(lambda x: encode_text(x, sbert_model))

In [12]:
def combine_features(text_embedding, numeric_data, sheds_data, care_experience):
    return np.hstack((text_embedding, numeric_data, sheds_data, [care_experience]))

In [13]:
dog_data['combined_embedding'] = dog_data.apply(
    lambda row: combine_features(
        row['embedding'],
        row[numeric_features].values,
        row[['Sheds Little', 'Sheds', 'Sheds A Lot']].values,
        row['Care Experience']
    ), axis=1
)

In [14]:
combined_embeddings = np.vstack(dog_data['combined_embedding'].values).astype('float32')
dimension = combined_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(combined_embeddings)

In [15]:
def search_dogs(query, cost=None, sheds=None, height=None, weight=None, care_experience=None, n_results=5):
    # Кодирование пользовательского запроса (текст)
    query_embedding = encode_text(query, sbert_model)

    # Нормализация введённых пользователем числовых параметров
    query_numeric = np.array([cost, height, weight]).reshape(1, -1)
    query_numeric = scaler.transform(query_numeric)

    # Обработка shedding данных в соответствии с пользовательским вводом
    sheds_data = np.array([0, 0, 0]).reshape(1, -1) # Reshape to 2D
    if sheds == 'Sheds Little':
        sheds_data = np.array([1, 0, 0]).reshape(1, -1) # Reshape to 2D
    elif sheds == 'Sheds':
        sheds_data = np.array([0, 1, 0]).reshape(1, -1) # Reshape to 2D
    elif sheds == 'Sheds A Lot':
        sheds_data = np.array([0, 0, 1]).reshape(1, -1) # Reshape to 2D

    # Комбинирование эмбеддинга запроса и нормализованных данных
    combined_query_embedding = np.hstack((query_embedding.reshape(1, -1), query_numeric, sheds_data, np.array([care_experience]).reshape(1, -1))).astype('float32') # Reshape to 2D

    # Поиск ближайших соседей в FAISS
    distances, indices = index.search(combined_query_embedding, n_results)

    # Преобразование расстояний на основе обратного расстояния
    relevance_percentages = relevance_to_percentage(distances[0])

    # Формирование результатов поиска
    results = []
    for i, idx in enumerate(indices[0]):
        dog = dog_data.iloc[idx]
        results.append({
            "Name": dog['Name'],
            "Description": dog['Description'],
            "Original Cost": denormalize(dog['Average Cost'], 'Average Cost'),
            "Original Height": denormalize(dog['Average Height'], 'Average Height'),
            "Original Weight": denormalize(dog['Average Weight'], 'Average Weight'),
            "Care Experience Needed": dog['Care Experience'],
            "Relevance Score": relevance_percentages[i]
        })
    return results

In [16]:
def asymmetric_search(query, cost=None, sheds=None, height=None, weight=None, care_experience=None, n_results=3):
    # Подготовка пользовательских параметров
    query_numeric = np.array([cost, height, weight]).reshape(1, -1)
    query_numeric = scaler.transform(query_numeric)

    # Обработка shedding данных в соответствии с пользовательским вводом
    sheds_data = np.array([0, 0, 0])
    if sheds == 'Sheds Little':
        sheds_data = np.array([1, 0, 0])
    elif sheds == 'Sheds':
        sheds_data = np.array([0, 1, 0])
    elif sheds == 'Sheds A Lot':
        sheds_data = np.array([0, 0, 1])

    # Подготовка всех данных для CrossEncoder
    results = []
    for idx, row in dog_data.iterrows():
        # Создаём объединённую строку для CrossEncoder на основе описания и числовых данных
        description = row['Description']
        combined_text = f"{description}. Цена: {denormalize(row['Average Cost'], 'Average Cost'):.2f}, " \
                        f"Рост: {denormalize(row['Average Height'], 'Average Height'):.2f}, " \
                        f"Вес: {denormalize(row['Average Weight'], 'Average Weight'):.2f}, " \
                        f"Sheds: {row['Sheds Little']}/{row['Sheds']}/{row['Sheds A Lot']}, " \
                        f"Требуется опыт ухода: {'Да' if row['Care Experience'] == 1 else 'Нет'}."

        # Оценка CrossEncoder
        score = cross_encoder.predict([(query, combined_text)])[0]
        results.append((score, idx))

    # Сортировка по оценкам релевантности и выбор топ-N
    sorted_results = sorted(results, key=lambda x: x[0], reverse=True)[:n_results]

    # Формирование выходных данных
    output = []
    for score, idx in sorted_results:
        dog = dog_data.iloc[idx]
        output.append({
            "Name": dog['Name'],
            "Description": dog['Description'],
            "Original Cost": denormalize(dog['Average Cost'], 'Average Cost'),
            "Original Height": denormalize(dog['Average Height'], 'Average Height'),
            "Original Weight": denormalize(dog['Average Weight'], 'Average Weight'),
            "Care Experience Needed": dog['Care Experience'],
            "Relevance Score": score
        })

    return output

In [17]:
query = "маленькая дружелюбная собака для квартиры"
cost = 45  # Цена
height = 50  # Рост
weight = 20  # Вес
care_experience = 1  # Нужен опыт ухода за собакой
sheds = 'Sheds Little'  # Выбор по shedding

In [18]:
results = search_dogs(query, cost, sheds, height, weight, care_experience, 5)

# Вывод результатов
for result in results:
    print(f"Порода: {result['Name']}")
    print(f"Цена: {result['Original Cost']:.2f}")
    print(f"Рост: {result['Original Height']:.2f}")
    print(f"Вес: {result['Original Weight']:.2f}")
    print(f"Опыт ухода: {'Да' if result['Care Experience Needed'] == 1 else 'Нет'}")
    print(f"Релевантность: {result['Relevance Score']}")
    print(f"Описание: {result['Description']}\n")

Порода: Фараонова собака
Цена: 67.50
Рост: 47.60
Вес: 22.50
Опыт ухода: Да
Релевантность: 1.0
Описание: фараоновый собака ить фарао хаунд род мальты считаться национальный достояние страна называться кельб тальфенек перевод означать кроличий собака мальтийский кроличий борзая обладать великолепный спортивный охотничий качествами использоваться охота зайцев кролик другой некрупный дичь идеально подходить компаньон питомец семьи элегантный мальтийский собака преследовать дикий зверь позрячему свой телосложение напоминать пёс поденко ибиценко дело абсолютно разный породы иметь никакой родственный связей единственный связывает статус борзых мальтийский борзая  врождённый аристократ интеллектуалы энергичные умные игривый общительный питомцы отличаться спокойным уравновесить характером достаточно упрямые обладать хороший мышлением ласковы феноменально миролюбивый открытый домочадец друг семьи хозяин испытывать тёплый чувство сильный привязанность надоедливые важно ощущать полноправный член с

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [19]:
results_asym = asymmetric_search(query, cost, sheds, height, weight, care_experience, 5)

# Вывод результатов асимметричного поиска
for result in results_asym:
    print(f"Порода: {result['Name']}")
    print(f"Цена: {result['Original Cost']:.2f}")
    print(f"Рост: {result['Original Height']:.2f}")
    print(f"Вес: {result['Original Weight']:.2f}")
    print(f"Опыт ухода: {'Да' if result['Care Experience Needed'] == 1 else 'Нет'}")
    print(f"Релевантность: {result['Relevance Score']}")
    print(f"Описание: {result['Description']}\n")

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Порода: Малая бельгийская собака
Цена: 45.00
Рост: 19.00
Вес: 4.00
Опыт ухода: Нет
Релевантность: 0.8291794061660767
Описание: малый бельгийский собака ить гриффон конюшенный гриффон  секция три декоративный порода собак входят давний время крохотный пёсик использоваться охрана дома уничтожение грызун питомец человек разный сословие  аристократ простолюдинов день являться домашний любимцами участвовать выставочный шоу разновидность гриффон произойти маленький жесткошёрстный собачка smousje существует когдато обитать окрестность бельгийский столицы отличаться цвет тип волосяной покрова брюссельский бельгийский гриффон характерный ус бородка морде отсутствовать брабанского гриффон  невероятно энергичный харизматический животное который любой сутки готовый проказничать играть натура наглый дерзким совершенно переносить одиночество начинать грустить отсутствие хозяина обожать любить свой маленький сердцем домочадец питомец чрезвычайно любвеобилен хозяин считаться самый главный свете ребёно

In [20]:
dog_data[dog_data['Name'] == 'Фараонова собака']

,Name,Link,Average Cost,Average Lifespan,Sheds Little,Sheds,Sheds A Lot,Average Litter Size,Average Height,Average Weight,Care Experience,Description,Image,embedding,combined_embedding
297,Фараонова собака,https://doge.ru/poroda/faraonovaya-sobaka,0.112745,16.0,1.0,0.0,0.0,6.5,-0.113079,-0.103905,1.0,фараоновый собака ить фарао хаунд род мальты с...,https://images.prismic.io/doge/74578921-5ba8-4...,"[0.016547062, 0.549206, -0.14716125, -0.001270...","[0.016547061502933502, 0.549206018447876, -0.1..."


In [21]:
sbert_model.save('sbert_model')
cross_encoder.save('cross_encoder')

In [22]:
np.save('dog_embeddings.npy', combined_embeddings)

In [23]:
dog_data.to_csv('cleaned_new.csv', index=False)

In [24]:
faiss.write_index(index, 'faiss_index.bin')

In [25]:
import joblib
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']